<a href="https://colab.research.google.com/github/alibakh62/DeepFake-Kaggle/blob/master/notebook/colab/DeepFake_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes



*   There are 300 frames in a 10 second video. 
    * Instead of getting each frames, we can average every _n_ frames.
*   The background in each video is non-relevant.
    * Need to add a face/body detection layer and crop the frames accrodingly.
* Need to capture movements between frames. Could use RNN, or feeding the difference between two consecutive frames
* Add the attention maps (Stehuawer paper) inside XceptionNet blocks.
* Frame sizes are `(1080, 1920)` and `(1920, 1080)`. Rotate the latter ones to make every frame into the same shape.



# Environment Setup

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
!pip install ipython-autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=e7295cd7b494f62f538db514ae31cb395b5f3bf8f11ca1e0931f6c20310c28ac
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [0]:
%load_ext autotime

In [4]:
%tensorflow_version 2.x

TensorFlow 2.x selected.
time: 6.15 ms


In [5]:
import tensorflow as tf
print(tf.__version__)

2.1.0
time: 7.47 s


In [6]:
!ls drive/My\ Drive/DeepFake

models	sample_submission.csv  test_videos  train_sample_videos
time: 848 ms


# utils

In [0]:
# play a video
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/drive/My Drive/DeepFake/train_sample_videos/aapnvogymq.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [0]:
# getting the number of frames in a video file
a = cv.VideoCapture('/content/drive/My Drive/DeepFake/train_sample_videos/cdaxixbosp.mp4')
a.get(cv.CAP_PROP_FRAME_COUNT)

# XceptionNet

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Activation
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D
#from tensorflow.keras.applications.imagenet_utils import _obtain_input_shape
from tensorflow.keras.utils import get_file

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
# from tensorflow.keras.utils import np_utils
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.optimizers import SGD
import os
import h5py
import matplotlib
import matplotlib.pyplot as plt
import random
#import cv2
from sklearn.preprocessing import LabelEncoder
from skimage import io, transform
from sklearn.metrics import accuracy_score
from scipy import misc
import numpy as np

time: 358 ms


In [8]:
# define the path for pretrained model
WEIGHTS_PATH = '/content/drive/My Drive/DeepFake/models/xception_weights_tf_dim_ordering_tf_kernels.h5'

time: 1.16 ms


In [9]:
#WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.4/xception_weights_tf_dim_ordering_tf_kernels.h5'

def Xception(nb_classes):

    # Determine proper input shape
#     input_shape = _obtain_input_shape(None, default_size=299, min_size=71, data_format='channels_last', include_top=False)

#     img_input = Input(shape=input_shape)
    img_input = Input(shape=(227,227,3))

    # Block 1
    x = Conv2D(32, (3, 3), strides=(2, 2), use_bias=False)(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    residual = Conv2D(128, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    # Block 2
    x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    # Block 2 Pool
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    residual = Conv2D(256, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    # Block 3
    x = Activation('relu')(x)
    x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    # Block 3 Pool
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    residual = Conv2D(728, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    # Block 4
    x = Activation('relu')(x)
    x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # Block 5 - 12
    for i in range(8):
        residual = x

        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)

        x = layers.add([x, residual])

    residual = Conv2D(1024, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    # Block 13
    x = Activation('relu')(x)
    x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(1024, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    # Block 13 Pool
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # Block 14
    x = SeparableConv2D(1536, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Block 14 part 2
    x = SeparableConv2D(2048, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Fully Connected Layer
    x = GlobalAveragePooling2D()(x)
    x = Dense(1000, activation='relu')(x)
    x = Dense(nb_classes, activation='softmax')(x)

    inputs = img_input

    # Create model
    model = Model(inputs, x, name='xception')

    # Download and cache the Xception weights file
    #weights_path = get_file('xception_weights.h5', WEIGHTS_PATH, cache_subdir='models')

    # load weights
    #model.load_weights(weights_path)

    return model


# """
#     Instantiate the model by using the following line of code

#     model = Xception()

# """

time: 156 ms


# Simple CNN

In [10]:
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from functools import partial

def simple_cnn(nb_classes):
    DefaultConv2D = partial(Conv2D,
                            kernel_size=3, activation='relu', padding="SAME")
    
    model = keras.models.Sequential([
        DefaultConv2D(filters=64, kernel_size=7, input_shape=[512, 512, 3]),
        MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=128),
        DefaultConv2D(filters=128),
        MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=256),
        DefaultConv2D(filters=256),
        MaxPooling2D(pool_size=2),
        Flatten(),
        Dense(units=128, activation='relu'),
        Dropout(0.5),
        Dense(units=64, activation='relu'),
        Dropout(0.5),
        Dense(units=nb_classes, activation='softmax'),
    ])

    return model

time: 8.44 ms


# DataLoader

In [11]:
import numpy as np
import pandas as pd
import os
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline 
import cv2 as cv

time: 583 ms


In [12]:
DATA_FOLDER = '/content/drive/My Drive/DeepFake'
TRAIN_SAMPLE_FOLDER = 'train_sample_videos'
TEST_FOLDER = 'test_videos'

print(f"Train samples: {len(os.listdir(os.path.join(DATA_FOLDER, TRAIN_SAMPLE_FOLDER)))}")
print(f"Test samples: {len(os.listdir(os.path.join(DATA_FOLDER, TEST_FOLDER)))}")

Train samples: 401
Test samples: 400
time: 15.7 ms


## Meta data

In [13]:
def get_meta_from_json(path):
    df = pd.read_json(os.path.join(DATA_FOLDER, path, json_file))
    df = df.T
    return df

train_list = list(os.listdir(os.path.join(DATA_FOLDER, TRAIN_SAMPLE_FOLDER)))
test_list = list(os.listdir(os.path.join(DATA_FOLDER, TEST_FOLDER)))
json_file = [file for file in train_list if  file.endswith('json')][0]
meta_train_df = get_meta_from_json(TRAIN_SAMPLE_FOLDER)
meta_train_df.head()

,label,split,original
aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
abarnvbtwb.mp4,REAL,train,None
abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


time: 1.32 s


In [14]:
meta_train_df["original"].value_counts()

meawmsgiti.mp4    6
atvmxvwyns.mp4    6
kgbkktcjxf.mp4    5
qeumxirsme.mp4    5
dzyuwjkjui.mp4    4
                 ..
qhkqqfznrg.mp4    1
sgjnvxvcpu.mp4    1
ygdgwyqyut.mp4    1
tfoxelmnjx.mp4    1
lietldeotq.mp4    1
Name: original, Length: 209, dtype: int64

time: 10.5 ms


In [15]:
meta_train_df.loc[meta_train_df.index == "aagfhgtpmv.mp4", "label"].values[0]

'FAKE'

time: 3.9 ms


# Train/Test Data

In [0]:
def walkdir(dirpath):
    for root, dirs, files in os.walk(dirpath):
        for name in files:
            if name != 'metadata.json':
                yield os.path.abspath(os.path.join(dirpath, name))

filescount = meta_train_df.index.nunique()

cnt = 1
X_train_full, y_train_full = [], []
for path in tqdm_notebook(walkdir(os.path.join(DATA_FOLDER, TRAIN_SAMPLE_FOLDER)), total=filescount):
    if cnt < 20:
        capture_image = cv.VideoCapture(path) 
        ret, frame = capture_image.read()
        if ret:
            image = np.array(tf.image.resize(frame, [512, 512]))
            X_train_full.append(image)
            label = 0
            if meta_train_df.loc[meta_train_df.index == name, "label"].values[0] == 'REAL':
                label = 1
            y_train_full.append(label)
        cnt += 1
    else:
        break

In [0]:
print(f"X_train size: {len(X_train_full)}, y_train size: {len(y_train_full)}")

X_train size: 19, y_train size: 19


In [0]:
# X_train_full = np.array(X_train_full)
# y_train_full = np.array(y_train_full)

split = 0.7
cutoff = int(split * len(X_train_full))

X_train, X_valid = X_train_full[:-cutoff], X_train_full[-cutoff:]
y_train, y_valid = y_train_full[:-cutoff], y_train_full[-cutoff:]

In [0]:
# X_train_full = np.array(X_train_full)
# y_train_full = np.array(y_train_full)

# split = 0.7
# cutoff = int(split * len(X_train))

# X_train, X_valid = X_train_full[:-cutoff], X_train_full[-cutoff:]
# y_train, y_valid = y_train_full[:-cutoff], y_train_full[-cutoff:]

# X_mean = X_train.mean(axis=0, keepdims=True)
# X_std = X_train.std(axis=0, keepdims=True) + 1e-7
# X_train = (X_train - X_mean) / X_std
# X_valid = (X_valid - X_mean) / X_std
# # X_test = (X_test - X_mean) / X_std

# X_train = X_train[..., np.newaxis]
# X_valid = X_valid[..., np.newaxis]
# # X_test = X_test[..., np.newaxis]

# Training Simple CNN

In [0]:
model = simple_cnn(1)

time: 146 ms


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 512, 512, 64)      9472      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 256, 256, 64)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 256, 256, 128)     73856     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 256, 256, 128)     147584    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 128, 128, 128)     0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 128, 128, 256)     295168    
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 128, 128, 256)     5

In [0]:
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

time: 40.5 ms


In [0]:
history = model.fit(X_train_, y_train_, batch_size=16, epochs=10, validation_data=[X_valid_, y_valid_])
# score = model.evaluate(X_test, y_test)

ValueError: ignored

time: 95 ms


In [0]:
MODELS_FOLDER = '/content/drive/My Drive/DeepFake/models'

In [0]:
MODEL_NAME = 'simple_cnn_test1.h5'
model.save_weights(os.path.join(MODELS_FOLDER, MODEL_NAME))

# Training XceptionNet

In [16]:
#@title Sample or All
sample = True #@param {type:"boolean"}
no_of_samples = 20 #@param {type:"integer"}

cnt_max = 2e10
if sample:
    cnt_max = no_of_samples

def walkdir(dirpath):
    for root, dirs, files in os.walk(dirpath):
        for name in files:
            if name != 'metadata.json':
                yield os.path.abspath(os.path.join(dirpath, name)), name

filescount = meta_train_df.index.nunique()

cnt = 1
X_train_full, y_train_full = [], []
for path, name in tqdm_notebook(walkdir(os.path.join(DATA_FOLDER, TRAIN_SAMPLE_FOLDER)), total=filescount):
    if cnt <= cnt_max:
        capture_image = cv.VideoCapture(path) 
        ret, frame = capture_image.read()
        if ret:
            image = np.array(tf.image.resize(frame, [227, 227]))
            X_train_full.append(image)
            label = 0
            if meta_train_df.loc[meta_train_df.index == name, "label"].values[0] == 'REAL':
                label = 1
            y_train_full.append(label)
        cnt += 1
    else:
        break

split = 0.3
cutoff = int(split * len(X_train_full))

X_train, X_valid = X_train_full[:-cutoff], X_train_full[-cutoff:]
y_train, y_valid = y_train_full[:-cutoff], y_train_full[-cutoff:]

from sklearn.utils.validation import column_or_1d
X_train_, y_train_ = np.array(X_train), column_or_1d(y_train)
X_valid_, y_valid_ = np.array(X_valid), column_or_1d(y_valid)

time: 55.2 s


In [17]:
from sklearn.utils.validation import column_or_1d
X_train_, y_train_ = np.array(X_train), column_or_1d(y_train)
X_valid_, y_valid_ = np.array(X_valid), column_or_1d(y_valid)

time: 8.09 ms


In [18]:
print(f"Number of training examples: {y_train_.shape}")
print(f"Number of validation examples: {y_valid_.shape}")
print(f"Number of reals in training: {y_train_.sum()}")
print(f"Number of reals in validation: {y_valid_.sum()}")

Number of training examples: (14,)
Number of validation examples: (5,)
Number of reals in training: 3
Number of reals in validation: 1
time: 3.38 ms


In [0]:
model_xception = Xception(1)

time: 1.6 s


In [0]:
model_xception.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 113, 113, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_40 (BatchNo (None, 113, 113, 32) 128         conv2d_6[0][0]                   
__________________________________________________________________________________________________
activation_35 (Activation)      (None, 113, 113, 32) 0           batch_normalization_40[0][0]     
___________________________________________________________________________________________

In [0]:
model_xception.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

time: 57.7 ms


In [0]:
from sklearn.utils.validation import column_or_1d
X_train_, y_train_ = np.array(X_train), column_or_1d(y_train)
X_valid_, y_valid_ = np.array(X_valid), column_or_1d(y_valid)
history_exception = model_xception.fit(X_train_, y_train_, batch_size=16, epochs=10, validation_data=[X_valid_, y_valid_])

Train on 280 samples, validate on 120 samples
Epoch 1/10
280/280 [==============================] - 15s 53ms/sample - loss: 12.5262 - accuracy: 0.1786 - val_loss: 11.8182 - val_accuracy: 0.2250
Epoch 2/10
280/280 [==============================] - 4s 16ms/sample - loss: 12.5262 - accuracy: 0.1786 - val_loss: 11.8182 - val_accuracy: 0.2250
Epoch 3/10
280/280 [==============================] - 4s 16ms/sample - loss: 12.5262 - accuracy: 0.1786 - val_loss: 11.8182 - val_accuracy: 0.2250
Epoch 4/10
280/280 [==============================] - 4s 16ms/sample - loss: 12.5262 - accuracy: 0.1786 - val_loss: 11.8182 - val_accuracy: 0.2250
Epoch 5/10
280/280 [==============================] - 4s 16ms/sample - loss: 12.5262 - accuracy: 0.1786 - val_loss: 11.8182 - val_accuracy: 0.2250
Epoch 6/10
280/280 [==============================] - 4s 16ms/sample - loss: 12.5262 - accuracy: 0.1786 - val_loss: 11.8182 - val_accuracy: 0.2250
Epoch 7/10
280/280 [==============================] - 4s 16ms/sample - 

In [0]:
MODELS_FOLDER = '/content/drive/My Drive/DeepFake/models'
MODEL_NAME = 'xception_test2_all_sample_training.h5'
model_xception.save_weights(os.path.join(MODELS_FOLDER, MODEL_NAME))

NameError: ignored

time: 25.6 ms


# Video Classification

***
### Storing frames of video

In [50]:
#@title Sample or All
sample = True #@param {type:"boolean"}
no_of_samples = 5 #@param {type:"integer"}

import math

def get_frames(cap, name, dirname, resize=None):
    frameRate = cap.get(5) #frame rate
    count = 0
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a different folder
            if resize is not None:
                # image = np.array(tf.image.resize(frame, resize))
                image = cv.resize(frame, resize)
            else:
                image = frame
            if not os.path.isdir(os.path.join(DATA_FOLDER, dirname)):
                os.mkdir(os.path.join(DATA_FOLDER, dirname))
            count += 1
            fn = os.path.splitext(name)[0] + f"_frame{count}.jpg"
            file_path = os.path.join(DATA_FOLDER, dirname, fn)
            cv.imwrite(file_path, image)
    cap.release()

cnt_max = 2e10
if sample:
    cnt_max = no_of_samples

def walkdir(dirpath):
    for root, dirs, files in os.walk(dirpath):
        for name in files:
            if name != 'metadata.json':
                yield os.path.abspath(os.path.join(dirpath, name)), name

filescount = meta_train_df.index.nunique()

cnt = 1
X_train_full, y_train_full = [], []
for path, name in tqdm_notebook(walkdir(os.path.join(DATA_FOLDER, TRAIN_SAMPLE_FOLDER)), total=filescount):
    if cnt <= cnt_max:
        cap = cv.VideoCapture(path)
        get_frames(cap, name, 'train_frames')
        cnt += 1

time: 16.1 s


In [0]:
import os
dirc = 'mydirpath'

filescount = len([i for i in os.listdir(dirc) if os.path.isfile(os.path.join(dirc, i))])
print(filescount)


In [0]:
TRAIN_FRAMES_FOLDER = 'train_frames'
X_train_full, y_train_full = [], []

for path, name in tqdm_notebook(walkdir(os.path.join(DATA_FOLDER, TRAIN_FRAMES_FOLDER)), total=filescount):
    